In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import auc
from sklearn import metrics

/opt/python/python35/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/opt/python/python35/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
path1 = '/axp/rim/imsads/dev/sagra39/IMS_Hackathon_17/input/derived/'
modelling_IP3CM_APP_device = pd.read_csv(path1 + 'modelling_IP3CM_APP_device.csv')

In [3]:
import copy
final_data = copy.deepcopy(modelling_IP3CM_APP_device)


pcn  = pd.DataFrame({'pcn':list(set(final_data['pcn']))})
train = pcn.sample(frac=0.7,random_state=1)
valid = set(pcn['pcn'])-set(train['pcn'])
valid = pd.DataFrame({'pcn':list(valid)})
valid = valid.sample(frac=0.67,random_state=1)

test  = set(pcn['pcn'])-set(train['pcn'])-set(valid['pcn'])
test  = pd.DataFrame({'pcn':list(test)})

print(train.shape)
print(valid.shape)
print(test.shape)

print(set(train['pcn'])&set(valid['pcn'])&set(test['pcn']))

train = train.merge(final_data, on=['pcn'], how='left')
valid = valid.merge(final_data, on=['pcn'], how='left')
test  = test.merge(final_data, on=['pcn'], how='left')

print(train.shape)
print(valid.shape)
print(test.shape)


#train.to_csv(path+'train.csv',index=False)
#valid.to_csv(path+'valid.csv',index=False)
#test.to_csv(path+'test.csv',index=False)

(74728, 1)
(21458, 1)
(10569, 1)
set()
(74728, 190)
(21458, 190)
(10569, 190)


In [4]:
list_drop = ['pcn', 'post_uni_visid', 'datetime', 'Market_cd_x', 'Market_cd_y']
train_x = train.drop(list_drop, axis=1)
train_y = train['Gamer_flag']

valid_x = valid.drop(list_drop, axis=1)
valid_y = valid['Gamer_flag']

test_x = test.drop(list_drop, axis=1)
test_y = test['Gamer_flag']


list_cat= []
for i in train_x:
    if train_x[i].dtypes == 'object':
        list_cat.append(i)
list_cat

['ip',
 'geo_region',
 'geo_city',
 'product_list',
 'user_agent',
 'Product_Color_Desc',
 'sub_channel',
 'ip_3',
 'DeviceType',
 'Osname',
 'Browsername',
 'primaryHardwareType',
 'model']

In [ ]:
train_x = train_x.fillna(-999)
valid_x = valid_x.fillna(-999)
test_x = test_x.fillna(-999)
for i in list_cat:
        train_x[i] = train_x[i].map(lambda x : str(x))
        valid_x[i] = valid_x[i].map(lambda x : str(x))
        test_x[i] = test_x[i].map(lambda x : str(x))
        
        le = preprocessing.LabelEncoder()
        train_x[i] = le.fit_transform(train_x[i])
        valid_x[i] = le.fit_transform(valid_x[i])
        test_x[i] = le.fit_transform(test_x[i])